## Data Description

This data represents the results of a large product testing study. For each `product_code` you are given a number of product `attributes` (fixed for the code) as well as a number of `measurement` values for each individual product, representing various lab testing methods. Each product is used in a simulated real-world environment experiment, and and absorbs a certain amount of fluid (`loading`) to see whether or not it fails. &nbsp;

Your task is to use the data to predict individual product failures of new codes with their individual lab test results.

## Evaluation

Submissions are evaluated on area under the ROC curve between the predicted probability and the observed target.<br>
The higher, the better

## Understanding

- Same product_code means they are the same products
- Same product consists of same attributes

## Blueprint

1. Numerize 'attribute_0' and 'attribute_1'
2. Drop the product code(A, B, C, D, E)
3. Apply PCA
4. Split into training and validating data
5. Apply ML models

### Import Packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn import metrics 

# Show all the columns and rows
pd.set_option("display.max_columns", None)  # columns
# pd.set_option("display.max_rows", None)   # rows

## 1. Data Loading

In [2]:
# Load dataset
data = pd.read_csv('train.csv') # training
te = pd.read_csv('test.csv')    # testing

print(data.shape)
data.head()

(26570, 26)


,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
0,0,A,80.10,material_7,material_8,9,5,7,8,4,18.040,12.518,15.748,19.292,11.739,20.155,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0
1,1,A,84.89,material_7,material_8,9,5,14,3,3,18.213,11.540,17.717,17.893,12.748,17.889,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0
2,2,A,82.43,material_7,material_8,9,5,12,1,5,18.057,11.652,16.738,18.240,12.718,18.288,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0
3,3,A,101.07,material_7,material_8,9,5,13,2,6,17.295,11.188,18.576,18.339,12.583,19.060,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0
4,4,A,188.06,material_7,material_8,9,5,9,2,8,19.346,12.950,16.990,15.746,11.306,18.093,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0


In [3]:
# Store 'id'
id = data.id

# Drop 'id'
data = data.drop(columns=['id'])

print(data.shape)
data.head()

(26570, 25)


,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
0,A,80.10,material_7,material_8,9,5,7,8,4,18.040,12.518,15.748,19.292,11.739,20.155,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0
1,A,84.89,material_7,material_8,9,5,14,3,3,18.213,11.540,17.717,17.893,12.748,17.889,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0
2,A,82.43,material_7,material_8,9,5,12,1,5,18.057,11.652,16.738,18.240,12.718,18.288,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0
3,A,101.07,material_7,material_8,9,5,13,2,6,17.295,11.188,18.576,18.339,12.583,19.060,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0
4,A,188.06,material_7,material_8,9,5,9,2,8,19.346,12.950,16.990,15.746,11.306,18.093,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0


## 2. Data Exploration

In [4]:
# Check data types and missing values
data.info()     # missing: loading, measurement_3 ~ measurement_17

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26570 entries, 0 to 26569
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   product_code    26570 non-null  object 
 1   loading         26320 non-null  float64
 2   attribute_0     26570 non-null  object 
 3   attribute_1     26570 non-null  object 
 4   attribute_2     26570 non-null  int64  
 5   attribute_3     26570 non-null  int64  
 6   measurement_0   26570 non-null  int64  
 7   measurement_1   26570 non-null  int64  
 8   measurement_2   26570 non-null  int64  
 9   measurement_3   26189 non-null  float64
 10  measurement_4   26032 non-null  float64
 11  measurement_5   25894 non-null  float64
 12  measurement_6   25774 non-null  float64
 13  measurement_7   25633 non-null  float64
 14  measurement_8   25522 non-null  float64
 15  measurement_9   25343 non-null  float64
 16  measurement_10  25270 non-null  float64
 17  measurement_11  25102 non-null 

In [5]:
# Check the distributions
print(data["failure"].value_counts())   # target
print(data["product_code"].value_counts())  # product

0    20921
1     5649
Name: failure, dtype: int64
C    5765
E    5343
B    5250
D    5112
A    5100
Name: product_code, dtype: int64


### - Treat Missing Values

In [ ]:
# # Opt A. Drop missing values
# data = data.dropna()

# # Opt B. Replace missing values with 0
# data = data.fillna(0)

# # Opt C. Replace missing values with the feature's mean
# data = data.fillna(data.mean())

In [ ]:
# # Check the distributions again
# print(data["failure"].value_counts())   # target
# print(data["product_code"].value_counts())  # product code

### - Objective Values

In [6]:
# Unique values in each column
print(data['product_code'].unique())
print(data['attribute_0'].unique())
print(data['attribute_1'].unique())

['A' 'B' 'C' 'D' 'E']
['material_7' 'material_5']
['material_8' 'material_5' 'material_6']


### - Int/Float Values

In [ ]:
data.describe()

## 3. Preprocessing

### (1) Numerize 'attribute_0' and 'attribute_1'

In [ ]:
# Apply function element-wise
numerized_attr01 = data.loc[:, ['attribute_0','attribute_1']].applymap(lambda x: int(x[9]))

In [ ]:
# Drop 'attribute_0', 'attribute_1' and concat
data = data.drop(columns=['attribute_0', 'attribute_1'])
data = pd.concat([numerized_attr01, data], axis=1)

In [ ]:
print(data.shape)
data.head()     # 12183(A), 26570(B), 26570(C)

In [ ]:
# Check attribute combinations for each product
def combinations(df):
    products = df['product_code'].unique()      # product codes
    attr = []       # list of the combination of attributes

    for product in products:
        attr = []
        subset = df.loc[df['product_code']==product, :]     # get subsets for each 'product code'

        attr.append(subset['attribute_0'].unique())
        attr.append(subset['attribute_1'].unique())
        attr.append(subset['attribute_2'].unique())
        attr.append(subset['attribute_3'].unique())

        print("Product",product, "consists of", attr)

In [ ]:
# Double check the combination
combinations(data)

### (2) Drop 'product_code'(A, B, C, D, E)

In [ ]:
# Drop 'product_code'
data = data.drop(columns=['product_code'])

print(data.shape)
data.head()

### (3)

data.corr()

### (3). Use 'loading' and 'attributes' only

In [ ]:
data = data.loc[:, ['attribute_0', 'attribute_1', 'attribute_2', 'attribute_3', 'loading', 'failure']]
data.info()

### (4) Treat Missing Values

In [ ]:
# # Opt A. Drop missing values
# data = data.dropna()

# # Opt B. Replace missing values with 0
# data = data.fillna(0)

# Opt C. Replace missing values with the feature's mean
data = data.fillna(data.mean())

In [ ]:
# Check the distributions again
print(data["failure"].value_counts())   # target
print(data.shape)

## 4. Apply ML models

In [ ]:
X = data.iloc[:, 0:5]  # all features
y = data.loc[:, 'failure']  # target

In [ ]:
# Split into training and validating
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=0, stratify=y)

### (1) Linear SVC

In [ ]:
from sklearn.svm import LinearSVC

LinSVC = LinearSVC(random_state=0)
LinSVC.fit(X_train, y_train)
y_LinSVC = LinSVC.predict(X_val)

In [ ]:
# Compute the area under the ROC curve
metrics.roc_auc_score(y_val, y_LinSVC)  # 0.5(A), 0.5(B), 0.5(C)

### (2) SGD Classifier

In [ ]:
from sklearn.linear_model import SGDClassifier

SGD = SGDClassifier(random_state=0)
SGD.fit(X_train, y_train)
y_SGD = SGD.predict(X_val)

In [ ]:
# Compute the area under the ROC curve
metrics.roc_auc_score(y_val, y_SGD)     # 0.540(A), 0.5(B), 0.504(C)

### (3) KNN Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

KNN = KNeighborsClassifier(n_neighbors=2)
KNN.fit(X_train, y_train)
y_KNN = KNN.predict(X_val)

In [ ]:
# Compute the area under the ROC curve
metrics.roc_auc_score(y_val, y_KNN)     # 0.504(A), 0.507(B), 0.507(C)

### (4) Kernel Approximation

In [ ]:
# check

### (5) SVC

In [ ]:
from sklearn.svm import SVC

SVC = SVC(random_state=0)
SVC.fit(X_train, y_train)
y_SVC = SVC.predict(X_val)

In [ ]:
# Compute the area under the ROC curve
metrics.roc_auc_score(y_val, y_SVC)     # 0.5(A), 0.5(B), 0.5(C)

### (6) Ensemble Classifiers

In [ ]:
from sklearn.ensemble import RandomForestRegressor

Ensb = RandomForestRegressor(max_depth=5, random_state=0)
Ensb.fit(X_train, y_train)
y_Ensb = SVC.predict(X_val)

In [ ]:
# Compute the area under the ROC curve
metrics.roc_auc_score(y_val, y_Ensb)    # 0.5(A), 0.5(B), 

## 5. Apply The Best-fit Model to The Testing Data